**Regras de associação**

**Algoritmo Apriori**

**Veja: https://www.youtube.com/watch?v=0v6PiOpsJi8**

In [11]:
!pip install apyori

In [ ]:
import pandas as pd
from apyori import apriori


In [3]:
#encoding = "cp1252", encoding='utf-8', encoding='latin1'etc
base= pd.read_csv('MercadoSim.csv', sep=';',encoding='cp1252', header=None)
base

,0,1,2,3,4,5,6
0,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
1,Leite,NaN,Cerveja,Pao,Manteiga,NaN,NaN
2,NaN,Cafe,NaN,Pao,Manteiga,NaN,NaN
3,Leite,Cafe,NaN,Pao,Manteiga,NaN,NaN
4,NaN,NaN,Cerveja,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,Manteiga,NaN,NaN
6,NaN,NaN,NaN,Pao,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,Feijao
8,NaN,NaN,NaN,NaN,NaN,Arroz,Feijao
9,NaN,NaN,NaN,NaN,NaN,Arroz,NaN


In [4]:
base.shape

(10, 7)

**Como estaremos treinando um modelo a priori, que recebe entradas em formato de lista, precisamos transformar o dataframe em uma lista de transações.**

In [6]:
transacoes = []
for i in range(len(base)):
  #print(i)
  #print(base_mercado1.values[i, 0])
  transacoes.append([str(base.values[i, j]) for j in range(base.shape[1])])

In [16]:
transacoes
transacoes = []
for i in range(len(base)):
    transacoes.append([str(base.values[i, j]) for j in range(base.shape[1]) if pd.notna(base.values[i, j])])


In [8]:
type(transacoes)

list

**Vamos chamar executar o algoritmo apriori e armazenar as regras obtidas**

In [17]:
regras = apriori(transacoes, min_support = 0.3, min_confidence = 0.8)

saida = list(regras)

# imprime total e regras
print(len(saida))
for regra in saida:
    for stat in regra.ordered_statistics:
        print(f"Regra: {list(stat.items_base)} -> {list(stat.items_add)} | suporte: {regra.support:.2f} | confiança: {stat.confidence:.2f} | lift: {stat.lift:.2f}")

4
Regra: ['Cafe'] -> ['Manteiga'] | suporte: 0.30 | confiança: 1.00 | lift: 2.00
Regra: ['Cafe'] -> ['Pao'] | suporte: 0.30 | confiança: 1.00 | lift: 2.00
Regra: ['Manteiga'] -> ['Pao'] | suporte: 0.40 | confiança: 0.80 | lift: 1.60
Regra: ['Pao'] -> ['Manteiga'] | suporte: 0.40 | confiança: 0.80 | lift: 1.60
Regra: ['Cafe'] -> ['Manteiga', 'Pao'] | suporte: 0.30 | confiança: 1.00 | lift: 2.50
Regra: ['Cafe', 'Manteiga'] -> ['Pao'] | suporte: 0.30 | confiança: 1.00 | lift: 2.00
Regra: ['Cafe', 'Pao'] -> ['Manteiga'] | suporte: 0.30 | confiança: 1.00 | lift: 2.00


In [11]:
print(saida[0])

RelationRecord(items=frozenset({'nan'}), support=1.0, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'nan'}), confidence=1.0, lift=1.0)])


In [12]:
print(saida[1])

RelationRecord(items=frozenset({'Manteiga', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Manteiga'}), confidence=1.0, lift=2.0)])


In [13]:
print(saida[2])

RelationRecord(items=frozenset({'Pao', 'Cafe'}), support=0.3, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Cafe'}), items_add=frozenset({'Pao'}), confidence=1.0, lift=2.0)])


**Para facilitar a visualização, veja como transformar isso em um dataframe**

Extraído de:

https://www.section.io/engineering-education/apriori-algorithm-in-python/

https://splunktool.com/understanding-apyoris-output

In [14]:
Antecedente = []
Consequente = []
suporte = []
confianca = []
lift = []
RegrasFinais = []

for resultado in saida:
  s = resultado[1]
  result_rules = resultado[2]
  for result_rule in result_rules:
    a = list(result_rule[0])
    b = list(result_rule[1])
    c = result_rule[2]
    l = result_rule[3]
    if 'nan' in a or 'nan' in b: continue
    if len(a) == 0 or len(b) == 0: continue
    Antecedente.append(a)
    Consequente.append(b)
    suporte.append(s)
    confianca.append(c)
    lift.append(l)
    RegrasFinais = pd.DataFrame({'Antecedente': Antecedente, 'Consequente': Consequente, 'suporte': suporte, 'confianca': confianca, 'lift': lift})

In [15]:
RegrasFinais

,Antecedente,Consequente,suporte,confianca,lift
0,[Cafe],[Manteiga],0.3,1.0,2.0
1,[Cafe],[Pao],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
4,[Cafe],"[Manteiga, Pao]",0.3,1.0,2.5
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
6,"[Pao, Cafe]",[Manteiga],0.3,1.0,2.0


**Podemos ordenar estas regras por uma métrica desejada**

In [16]:
RegrasFinais.sort_values(by='lift', ascending =False)

,Antecedente,Consequente,suporte,confianca,lift
4,[Cafe],"[Manteiga, Pao]",0.3,1.0,2.5
0,[Cafe],[Manteiga],0.3,1.0,2.0
1,[Cafe],[Pao],0.3,1.0,2.0
5,"[Manteiga, Cafe]",[Pao],0.3,1.0,2.0
6,"[Pao, Cafe]",[Manteiga],0.3,1.0,2.0
2,[Manteiga],[Pao],0.4,0.8,1.6
3,[Pao],[Manteiga],0.4,0.8,1.6
